In [1]:
!pip install -q underthesea
!pip install -q rouge-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 30.2 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer, ViTModel, ViTImageProcessor
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image
from tqdm import tqdm
from torch.optim import AdamW
from torch.cuda.amp import GradScaler, autocast
import nltk
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer
import random
from collections import defaultdict
import pickle

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
TRAIN_BATCH_SIZE = 4
VAL_BATCH_SIZE = 1
MAX_LEN = 100
EPOCHS = 2
ACCUMULATION_STEPS = 4

2025-05-17 03:55:26.506450: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747454126.694871      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747454126.749768      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
def split_into_tiles(image, tile_size=448):
    img_width, img_height = image.size
    tiles = []
    for y in range(0, img_height, tile_size):
        for x in range(0, img_width, tile_size):
            box = (x, y, min(x + tile_size, img_width), min(y + tile_size, img_height))
            tile = image.crop(box)
            if tile.size != (tile_size, tile_size):
                padded_tile = Image.new('RGB', (tile_size, tile_size), (0, 0, 0))
                padded_tile.paste(tile, (0, 0))
                tile = padded_tile
            tiles.append(tile)
    return tiles

In [4]:
def preprocess_images(df, image_processor, vision_encoder, max_tiles=4, cache_dir="visual_cache"):
    os.makedirs(cache_dir, exist_ok=True)
    visual_cache = {}
    grouped = df.groupby('image_path').agg({
        'question': list,
        'answer': list,
        'image_id': 'first',
        'index': 'first'
    }).reset_index()

    vision_encoder.eval()
    with torch.no_grad():
        for _, row in tqdm(grouped.iterrows(), total=len(grouped), desc="Tiền xử lý ảnh"):
            image_path = row['image_path']
            cache_path = os.path.join(cache_dir, f"{row['image_id']}.pkl")
            if os.path.exists(cache_path):
                continue

            try:
                image = Image.open(image_path).convert('RGB')
                tiles = split_into_tiles(image, tile_size=448)[:max_tiles]
                processed_tiles = [image_processor(tile, return_tensors="pt")['pixel_values'].squeeze(0) for tile in tiles]
                visual_tokens = torch.stack(processed_tiles).to(device)  # [num_tiles, 3, 448, 448]
                visual_features = vision_encoder(pixel_values=visual_tokens).last_hidden_state  # [num_tiles, seq_len, 768]
                visual_features = visual_features.mean(dim=1)  # [num_tiles, 768]
                
                # Kiểm tra kích thước
                assert visual_features.shape == (max_tiles, 768), f"Kích thước visual_features không đúng: {visual_features.shape}"
                
                visual_cache[image_path] = visual_features.cpu()
                with open(cache_path, 'wb') as f:
                    pickle.dump(visual_features.cpu(), f)
            except Exception as e:
                print(f"Lỗi xử lý ảnh {image_path}: {e}")
                visual_cache[image_path] = torch.zeros((max_tiles, 768))

    return visual_cache

In [5]:
class VQADataset(Dataset):
    def __init__(self, df, tokenizer, cache_dir="visual_cache", max_len=100, max_tiles=4):
        self.df = df.groupby('image_path').agg({
            'question': list,
            'answer': list,
            'image_id': 'first',
            'index': 'first'
        }).reset_index()
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.cache_dir = cache_dir
        self.max_tiles = max_tiles
        self.visual_cache = defaultdict(lambda: None)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = row['image_path']
        questions = row['question']
        answers = row['answer']
        image_id = row['image_id']

        if self.visual_cache[image_path] is None:
            cache_path = os.path.join(self.cache_dir, f"{image_id}.pkl")
            if os.path.exists(cache_path):
                with open(cache_path, 'rb') as f:
                    visual_tokens = pickle.load(f)
                self.visual_cache[image_path] = visual_tokens
            else:
                print(f"Không tìm thấy cache cho ảnh {image_path}")
                visual_tokens = torch.zeros((self.max_tiles, 768))

        visual_tokens = self.visual_cache[image_path].to(device)  # [num_tiles, 768]

        idx_qa = random.randint(0, len(questions) - 1)
        question = questions[idx_qa]
        answer = answers[idx_qa]

        question_encoding = self.tokenizer(question, max_length=self.max_len, padding='max_length', truncation=True, return_tensors='pt')
        answer_encoding = self.tokenizer(answer, max_length=self.max_len, padding='max_length', truncation=True, return_tensors='pt')

        return {
            'visual_tokens': visual_tokens,
            'question_ids': question_encoding['input_ids'].squeeze(),
            'question_mask': question_encoding['attention_mask'].squeeze(),
            'answer_ids': answer_encoding['input_ids'].squeeze(),
            'answer_mask': answer_encoding['attention_mask'].squeeze()
        }

In [6]:
class Vintern1BModel(nn.Module):
    def __init__(self, max_tiles=4):
        super(Vintern1BModel, self).__init__()
        self.max_tiles = max_tiles
        self.mlp_projector = nn.Sequential(
            nn.Linear(768, 2048),
            nn.ReLU(),
            nn.Linear(2048, 4096)
        )
        self.language_model = AutoModel.from_pretrained("Qwen/Qwen2-0.5B-Instruct")

    def forward(self, visual_tokens, question_ids, question_mask, answer_ids=None):
        batch_size = question_ids.shape[0]
        # visual_tokens: [batch_size, num_tiles, 768]
        if visual_tokens.dim() == 2:  # [num_tiles, 768] -> [1, num_tiles, 768]
            visual_tokens = visual_tokens.unsqueeze(0)
        if visual_tokens.shape[0] != batch_size:
            visual_tokens = visual_tokens.repeat(batch_size, 1, 1)
        
        projected_features = self.mlp_projector(visual_tokens)  # [batch_size, num_tiles, 4096]
        inputs_embeds = self.language_model.get_input_embeddings()(question_ids)  # [batch_size, seq_len, 4096]
        combined_embeds = torch.cat([projected_features.mean(dim=1, keepdim=True), inputs_embeds], dim=1)  # [batch_size, 1 + seq_len, 4096]
        outputs = self.language_model(inputs_embeds=combined_embeds, attention_mask=question_mask, labels=answer_ids if answer_ids is not None else None)
        return outputs

In [7]:
df_train = pd.read_csv("/kaggle/input/data-nlp-bai-2/df_train.csv")
df_train = df_train.sort_values(by=["image_id"])
df_train = df_train.iloc[:10000]
df_train

,index,image_id,image_path,question,answer
5430,4,0,/kaggle/input/data-nlp-bai-2/img/img/0.jpg,sách này phù_hợp với trình_độ học_vấn nào,cuốn sách dành cho học_sinh lớp 12
130346,14,0,/kaggle/input/data-nlp-bai-2/img/img/0.jpg,ấn_phẩm này được xuất_bản bởi tổ_chức nào,cuốn sách được xuất_bản bởi nhà_xuất_bản giáo_...
172135,13,0,/kaggle/input/data-nlp-bai-2/img/img/0.jpg,nhà_xuất_bản nào phát_hành cuốn sách này,cuốn sách được xuất_bản bởi nhà_xuất_bản giáo_...
149545,7,0,/kaggle/input/data-nlp-bai-2/img/img/0.jpg,môn_học được đề_cập đến trong cuốn sách ở ảnh ...,cuốn sách trong ảnh thuộc bộ_môn_ngữ văn
213389,5,0,/kaggle/input/data-nlp-bai-2/img/img/0.jpg,đối_tượng độc_giả của cuốn sách này là ai,cuốn sách dành cho học_sinh lớp 12
...,...,...,...,...,...
102697,269206,833,/kaggle/input/data-nlp-bai-2/img/img/833.jpg,trang sách kết_thúc bằng câu hỏi gợi_mở điều gì,câu hỏi cuối_cùng của trang sách đặt ra vấn_đề...
33054,269205,833,/kaggle/input/data-nlp-bai-2/img/img/833.jpg,vấn_đề nào được đề_cập ở câu hỏi khép lại tran...,câu hỏi cuối_cùng của trang sách đặt ra vấn_đề...
148169,269212,833,/kaggle/input/data-nlp-bai-2/img/img/833.jpg,việc bán_thân làm nô_lệ được đề_cập ra sao tro...,pháp_luật nhà lê hạn_chế nghiêm_ngặt việc bán ...
139086,269207,833,/kaggle/input/data-nlp-bai-2/img/img/833.jpg,nêu những khó_khăn mà nông_dân thời lê_sơ phải...,nông_dân thời lê_sơ phải đối_mặt với nhiều khó...


In [8]:
df_test = pd.read_csv("/kaggle/input/data-nlp-bai-2/df_test.csv")
df_test = df_test.sort_values(by=["image_id"])
df_test = df_test.iloc[:100]
df_test

,index,image_id,image_path,question,answer
20950,12,0,/kaggle/input/data-nlp-bai-2/img/img/0.jpg,cuốn sách do đơn_vị nào xuất_bản,cuốn sách được xuất_bản bởi nhà_xuất_bản giáo_...
25785,11,0,/kaggle/input/data-nlp-bai-2/img/img/0.jpg,đây là lần tái_bản thứ mấy của cuốn sách này,cuốn sách được in lần thứ ba
20374,6,0,/kaggle/input/data-nlp-bai-2/img/img/0.jpg,cuốn sách trong ảnh thuộc bộ_môn_học nào,cuốn sách trong ảnh thuộc bộ_môn_ngữ văn
24192,22,1,/kaggle/input/data-nlp-bai-2/img/img/1.jpg,ví_dụ 2 câu b mô_tả số phức_nào với độ dài bằn...,câu b cho biết số phức 2 cosπ isinπ có modun b...
447,26,1,/kaggle/input/data-nlp-bai-2/img/img/1.jpg,câu c của ví_dụ 2 xác_định số phức nào với giá...,câu c cho biết số phức_cos π 2 isin π 2 có mod...
...,...,...,...,...,...
32288,107525,27,/kaggle/input/data-nlp-bai-2/img/img/27.jpg,giá_trị số trang nằm ở vị_trí góc trên bên trá...,số trang được ghi ở góc trái của trang sách là...
42236,113189,28,/kaggle/input/data-nlp-bai-2/img/img/28.jpg,bài thơ có sử_dụng những biện_pháp nghệ_thuật nào,bài thơ sử_dụng nhiều biện_pháp nghệ_thuật như...
1570,113196,28,/kaggle/input/data-nlp-bai-2/img/img/28.jpg,bài thơ này chứa_đựng ý_nghĩa sâu_sắc nào,thông_qua bài thơ_tác_giả muốn gửi_gắm thông_đ...
46056,113191,28,/kaggle/input/data-nlp-bai-2/img/img/28.jpg,bài thơ đã vận_dụng những phương_thức biểu_đạt...,bài thơ sử_dụng nhiều biện_pháp nghệ_thuật như...


In [9]:
# Khởi tạo
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B-Instruct")
image_processor = ViTImageProcessor.from_pretrained("OpenGVLab/InternViT-300M-448px")
vision_encoder = ViTModel.from_pretrained("OpenGVLab/InternViT-300M-448px").to(device)

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/748 [00:00<?, ?B/s]

You are using a model of type intern_vit_6b to instantiate a model of type vit. This is not supported for all configurations of models and can yield errors.


model.safetensors:   0%|          | 0.00/608M [00:00<?, ?B/s]

Some weights of ViTModel were not initialized from the model checkpoint at OpenGVLab/InternViT-300M-448px and are newly initialized: ['embeddings.cls_token', 'embeddings.patch_embeddings.projection.bias', 'embeddings.patch_embeddings.projection.weight', 'embeddings.position_embeddings', 'encoder.layer.0.attention.attention.key.bias', 'encoder.layer.0.attention.attention.key.weight', 'encoder.layer.0.attention.attention.query.bias', 'encoder.layer.0.attention.attention.query.weight', 'encoder.layer.0.attention.attention.value.bias', 'encoder.layer.0.attention.attention.value.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.layernorm_after.bias', 'encoder.layer.0.layernorm_after.weight', 'encoder.layer.0.layernorm_before.bias', 'encoder.layer.0.layernorm_before.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output

In [10]:
import shutil
shutil.rmtree("visual_cache_train", ignore_errors=True)
shutil.rmtree("visual_cache_test", ignore_errors=True)

preprocess_images(df_train, image_processor, vision_encoder, max_tiles=4, cache_dir="visual_cache_train")
preprocess_images(df_test, image_processor, vision_encoder, max_tiles=4, cache_dir="visual_cache_test")

Tiền xử lý ảnh:   0%|          | 1/834 [00:01<14:50,  1.07s/it]

Lỗi xử lý ảnh /kaggle/input/data-nlp-bai-2/img/img/0.jpg: Kích thước visual_features không đúng: torch.Size([4, 1024])
Lỗi xử lý ảnh /kaggle/input/data-nlp-bai-2/img/img/1.jpg: Kích thước visual_features không đúng: torch.Size([4, 1024])


Tiền xử lý ảnh:   0%|          | 3/834 [00:01<06:35,  2.10it/s]

Lỗi xử lý ảnh /kaggle/input/data-nlp-bai-2/img/img/10.jpg: Kích thước visual_features không đúng: torch.Size([4, 1024])


Tiền xử lý ảnh:   0%|          | 4/834 [00:02<06:33,  2.11it/s]

...

Tiền xử lý ảnh:  97%|█████████▋| 28/29 [00:13<00:00,  2.12it/s]

Lỗi xử lý ảnh /kaggle/input/data-nlp-bai-2/img/img/8.jpg: Kích thước visual_features không đúng: torch.Size([4, 1024])


Tiền xử lý ảnh: 100%|██████████| 29/29 [00:13<00:00,  2.13it/s]

Lỗi xử lý ảnh /kaggle/input/data-nlp-bai-2/img/img/9.jpg: Kích thước visual_features không đúng: torch.Size([4, 1024])


{'/kaggle/input/data-nlp-bai-2/img/img/0.jpg': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 '/kaggle/input/data-nlp-bai-2/img/img/1.jpg': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 '/kaggle/input/data-nlp-bai-2/img/img/10.jpg': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 '/kaggle/input/data-nlp-bai-2/img/img/11.jpg': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 '/kaggle/input/data-nlp-bai-2/img/img/12.jpg': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,

In [11]:
# Tạo dataset và dataloader
train_dataset = VQADataset(df_train, tokenizer, cache_dir="visual_cache_train")
test_dataset = VQADataset(df_test, tokenizer, cache_dir="visual_cache_test")
train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=VAL_BATCH_SIZE, shuffle=False)

# Huấn luyện
model = Vintern1BModel(max_tiles=4).to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)
scaler = torch.amp.GradScaler('cuda')

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [12]:
model.train()
for epoch in range(EPOCHS):
    total_loss = 0
    optimizer.zero_grad()
    for i, batch in enumerate(tqdm(train_loader)):
        visual_tokens = batch['visual_tokens'].to(device)  # [batch_size, num_tiles, 768]
        question_ids = batch['question_ids'].to(device)
        question_mask = batch['question_mask'].to(device)
        answer_ids = batch['answer_ids'].to(device)
        
        with torch.amp.autocast('cuda'):
            outputs = model(visual_tokens, question_ids, question_mask, answer_ids)
            loss = outputs.loss / ACCUMULATION_STEPS
        
        scaler.scale(loss).backward()
        
        if (i + 1) % ACCUMULATION_STEPS == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
        
        total_loss += loss.item() * ACCUMULATION_STEPS
    
    print(f"Kỷ nguyên {epoch+1}/{EPOCHS}, Mất mát: {total_loss/len(train_loader)}")

# Lưu mô hình
torch.save(model.state_dict(), "vintern1b_model.pth")

  0%|          | 0/209 [00:00<?, ?it/s]

Không tìm thấy cache cho ảnh /kaggle/input/data-nlp-bai-2/img/img/824.jpg


AttributeError: 'NoneType' object has no attribute 'to'

In [ ]:
def evaluate_model(model, dataloader, tokenizer):
    model.eval()
    predictions, references = [], []
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    
    with torch.no_grad():
        for batch in tqdm(dataloader):
            visual_tokens = batch['visual_tokens'].to(device)
            question_ids = batch['question_ids'].to(device)
            question_mask = batch['question_mask'].to(device)
            
            outputs = model(visual_tokens, question_ids, question_mask)
            pred_ids = torch.argmax(outputs.logits, dim=-1)
            pred_text = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
            ref_text = tokenizer.batch_decode(batch['answer_ids'], skip_special_tokens=True)
            
            predictions.extend(pred_text)
            references.extend(ref_text)
    
    bleu_score = corpus_bleu([[ref.split()] for ref in references], [pred.split() for pred in predictions])
    rouge_scores = [scorer.score(ref, pred) for ref, pred in zip(references, predictions)]
    avg_rouge1 = sum([score['rouge1'].fmeasure for score in rouge_scores]) / len(rouge_scores)
    avg_rougeL = sum([score['rougeL'].fmeasure for score in rouge_scores]) / len(rouge_scores)
    
    return bleu_score, avg_rouge1, avg_rougeL

bleu, rouge1, rougeL = evaluate_model(model, test_loader, tokenizer)
print(f"BLEU: {bleu}, ROUGE-1: {rouge1}, ROUGE-L: {rougeL}")

In [17]:
!pip install -q underthesea
!pip install -q rouge-score
!pip install -q nltk

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer, ViTModel, ViTImageProcessor
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image
from tqdm import tqdm
from torch.optim import AdamW
import torch.amp
import nltk
nltk.download('punkt')
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer
import random
from collections import defaultdict
import pickle
import shutil

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
TRAIN_BATCH_SIZE = 4
VAL_BATCH_SIZE = 1
MAX_LEN = 100
EPOCHS = 2
ACCUMULATION_STEPS = 4
MAX_TILES = 4

def split_into_tiles(image, tile_size=448):
    img_width, img_height = image.size
    tiles = []
    for y in range(0, img_height, tile_size):
        for x in range(0, img_width, tile_size):
            box = (x, y, min(x + tile_size, img_width), min(y + tile_size, img_height))
            tile = image.crop(box)
            if tile.size != (tile_size, tile_size):
                padded_tile = Image.new('RGB', (tile_size, tile_size), (0, 0, 0))
                padded_tile.paste(tile, (0, 0))
                tile = padded_tile
            tiles.append(tile)
    return tiles

def preprocess_images(df, image_processor, vision_encoder, max_tiles=4, cache_dir="visual_cache"):
    os.makedirs(cache_dir, exist_ok=True)
    visual_cache = {}
    grouped = df.groupby('image_path').agg({
        'question': list,
        'answer': list,
        'image_id': 'first',
        'index': 'first'
    }).reset_index()

    vision_encoder.eval()
    with torch.no_grad():
        for _, row in tqdm(grouped.iterrows(), total=len(grouped), desc="Tiền xử lý ảnh"):
            image_path = row['image_path']
            cache_path = os.path.join(cache_dir, f"{row['image_id']}.pkl")
            if os.path.exists(cache_path):
                continue

            try:
                image = Image.open(image_path).convert('RGB')
                tiles = split_into_tiles(image, tile_size=448)[:max_tiles]
                processed_tiles = [image_processor(tile, return_tensors="pt")['pixel_values'].squeeze(0) for tile in tiles]
                visual_tokens = torch.stack(processed_tiles).to(device)
                visual_features = vision_encoder(pixel_values=visual_tokens).last_hidden_state
                visual_features = visual_features.mean(dim=1)
                
                assert visual_features.shape == (max_tiles, 1024), f"Kích thước visual_features không đúng: {visual_features.shape}"
                
                visual_cache[image_path] = visual_features.cpu()
                with open(cache_path, 'wb') as f:
                    pickle.dump(visual_features.cpu(), f)
            except Exception as e:
                print(f"Lỗi xử lý ảnh {image_path}: {e}")
                visual_cache[image_path] = torch.zeros((max_tiles, 1024))

    return visual_cache

class VQADataset(Dataset):
    def __init__(self, df, tokenizer, cache_dir="visual_cache", max_len=100, max_tiles=4):
        self.df = df.groupby('image_path').agg({
            'question': list,
            'answer': list,
            'image_id': 'first',
            'index': 'first'
        }).reset_index()
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.cache_dir = cache_dir
        self.max_tiles = max_tiles
        self.visual_cache = defaultdict(lambda: None)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = row['image_path']
        questions = row['question']
        answers = row['answer']
        image_id = row['image_id']

        if self.visual_cache[image_path] is None:
            cache_path = os.path.join(self.cache_dir, f"{image_id}.pkl")
            if os.path.exists(cache_path):
                with open(cache_path, 'rb') as f:
                    visual_tokens = pickle.load(f)
                self.visual_cache[image_path] = visual_tokens
            else:
                print(f"Không tìm thấy cache cho ảnh {image_path}")
                visual_tokens = torch.zeros((self.max_tiles, 1024))

        visual_tokens = self.visual_cache[image_path].to(device)

        idx_qa = random.randint(0, len(questions) - 1)
        question = questions[idx_qa]
        answer = answers[idx_qa]

        question_encoding = self.tokenizer(question, max_length=self.max_len, padding='max_length', truncation=True, return_tensors='pt')
        answer_encoding = self.tokenizer(answer, max_length=self.max_len, padding='max_length', truncation=True, return_tensors='pt')

        return {
            'visual_tokens': visual_tokens,
            'question_ids': question_encoding['input_ids'].squeeze(),
            'question_mask': question_encoding['attention_mask'].squeeze(),
            'answer_ids': answer_encoding['input_ids'].squeeze(),
            'answer_mask': answer_encoding['attention_mask'].squeeze()
        }

class Vintern1BModel(nn.Module):
    def __init__(self, max_tiles=4):
        super(Vintern1BModel, self).__init__()
        self.max_tiles = max_tiles
        self.mlp_projector = nn.Sequential(
            nn.Linear(1024, 2048),
            nn.ReLU(),
            nn.Linear(2048, 896)  # Đổi từ 4096 thành 896
        )
        self.language_model = AutoModel.from_pretrained("Qwen/Qwen2-0.5B-Instruct")

    def forward(self, visual_tokens, question_ids, question_mask, answer_ids=None):
        batch_size = question_ids.shape[0]
        if visual_tokens.dim() == 2:
            visual_tokens = visual_tokens.unsqueeze(0)
        if visual_tokens.shape[0] != batch_size:
            visual_tokens = visual_tokens.repeat(batch_size, 1, 1)
        
        projected_features = self.mlp_projector(visual_tokens)  # [batch_size, num_tiles, 896]
        inputs_embeds = self.language_model.get_input_embeddings()(question_ids)  # [batch_size, seq_len, 896]
        combined_embeds = torch.cat([projected_features.mean(dim=1, keepdim=True), inputs_embeds], dim=1)
        outputs = self.language_model(inputs_embeds=combined_embeds, attention_mask=question_mask, labels=answer_ids if answer_ids is not None else None)
        return outputs

df_train = pd.read_csv("/kaggle/input/data-nlp-bai-2/df_train.csv")
df_train = df_train.sort_values(by=["image_id"])
df_train = df_train.iloc[:10000]

df_test = pd.read_csv("/kaggle/input/data-nlp-bai-2/df_test.csv")
df_test = df_test.sort_values(by=["image_id"])
df_test = df_test.iloc[:100]

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B-Instruct")
image_processor = ViTImageProcessor.from_pretrained("OpenGVLab/InternViT-300M-448px")
vision_encoder = ViTModel.from_pretrained("OpenGVLab/InternViT-300M-448px").to(device)

# Xóa cache cũ và tiền xử lý lại
shutil.rmtree("visual_cache_train", ignore_errors=True)
shutil.rmtree("visual_cache_test", ignore_errors=True)
preprocess_images(df_train, image_processor, vision_encoder, max_tiles=MAX_TILES, cache_dir="visual_cache_train")
preprocess_images(df_test, image_processor, vision_encoder, max_tiles=MAX_TILES, cache_dir="visual_cache_test")

train_dataset = VQADataset(df_train, tokenizer, cache_dir="visual_cache_train", max_len=MAX_LEN, max_tiles=MAX_TILES)
test_dataset = VQADataset(df_test, tokenizer, cache_dir="visual_cache_test", max_len=MAX_LEN, max_tiles=MAX_TILES)
train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=VAL_BATCH_SIZE, shuffle=False)

model = Vintern1BModel(max_tiles=MAX_TILES).to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)
scaler = torch.amp.GradScaler('cuda')

model.train()
for epoch in range(EPOCHS):
    total_loss = 0
    optimizer.zero_grad()
    for i, batch in enumerate(tqdm(train_loader)):
        visual_tokens = batch['visual_tokens'].to(device)
        question_ids = batch['question_ids'].to(device)
        question_mask = batch['question_mask'].to(device)
        answer_ids = batch['answer_ids'].to(device)
        
        with torch.amp.autocast('cuda'):
            outputs = model(visual_tokens, question_ids, question_mask, answer_ids)
            loss = outputs.loss / ACCUMULATION_STEPS
        
        scaler.scale(loss).backward()
        
        if (i + 1) % ACCUMULATION_STEPS == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
        
        total_loss += loss.item() * ACCUMULATION_STEPS
    
    print(f"Kỷ nguyên {epoch+1}/{EPOCHS}, Mất mát: {total_loss/len(train_loader)}")

torch.save(model.state_dict(), "vintern1b_model.pth")

def evaluate_model(model, dataloader, tokenizer):
    model.eval()
    predictions, references = [], []
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    
    with torch.no_grad():
        for batch in tqdm(dataloader):
            visual_tokens = batch['visual_tokens'].to(device)
            question_ids = batch['question_ids'].to(device)
            question_mask = batch['question_mask'].to(device)
            
            outputs = model(visual_tokens, question_ids, question_mask)
            pred_ids = torch.argmax(outputs.logits, dim=-1)
            pred_text = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
            ref_text = tokenizer.batch_decode(batch['answer_ids'], skip_special_tokens=True)
            
            predictions.extend(pred_text)
            references.extend(ref_text)
    
    bleu_score = corpus_bleu([[ref.split()] for ref in references], [pred.split() for pred in predictions])
    rouge_scores = [scorer.score(ref, pred) for ref, pred in zip(references, predictions)]
    avg_rouge1 = sum([score['rouge1'].fmeasure for score in rouge_scores]) / len(rouge_scores)
    avg_rougeL = sum([score['rougeL'].fmeasure for score in rouge_scores]) / len(rouge_scores)
    
    return bleu_score, avg_rouge1, avg_rougeL

bleu, rouge1, rougeL = evaluate_model(model, test_loader, tokenizer)
print(f"BLEU: {bleu}, ROUGE-1: {rouge1}, ROUGE-L: {rougeL}")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[nltk_data] Downloading pack

RuntimeError: The expanded size of the tensor (101) must match the existing size (100) at non-singleton dimension 3.  Target sizes: [4, 14, 101, 101].  Tensor sizes: [4, 1, 101, 100]